<a href="https://www.kaggle.com/code/pratyushraj07/inference-effnet?scriptVersionId=121184721" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
try:
    import pylibjpeg
except:
    !pip install -q /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
    !pip install -q /kaggle/input/rsna-bcd-whl-ds/python_gdcm-3.0.20-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    # !pip install -q /kaggle/input/rsna-bcd-whl-ds/pylibjpeg-1.4.0-py3-none-any.whl
    !pip install -q /kaggle/input/rsna-bcd-whl-ds/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
""""
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)
        k  = nn.Sigmoid()
        out = k(out)
        out = torch.squeeze(out,1)
        loss = F.binary_cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)
        k  = nn.Sigmoid()
        out = k(out)
        out = torch.squeeze(out,1)
        #print(out)
        loss = F.binary_cross_entropy(out, labels)
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
         print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
#     _, preds = torch.max(outputs, dim=1)
#     print(outputs)
#     print(labels)
    for i in range(len(outputs)):
        if (outputs[i]>0.5):
            outputs[i] = 1
        else:
            outputs[i] = 0
    return torch.tensor(torch.sum(outputs == labels).item() / len(outputs))
"""    

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:

device = get_default_device()

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import cv2
import dicomsdl

In [ ]:
MODELS_PATH = '/kaggle/input/wandb-models/models'
DCM_TEST_IMAGES_PATH = f'/kaggle/input/rsna-breast-cancer-detection/test_images'

In [ ]:
import timm

class EfficientNet_b0(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = timm.create_model("efficientnet_b0")
        # Replace last layer
        num_ftrs = self.network.classifier.in_features
#         self.network.classifier = nn.Linear(num_ftrs,3)7
        self.network.classifier = torch.nn.Sequential(torch.nn.Linear(num_ftrs,1))
    
    def forward(self, xb):
        return self.network(xb)
    

In [ ]:
#model

In [ ]:
effnet_b0 = EfficientNet_b0()
effnet_b0 = effnet_b0.to(device)
model = effnet_b0

In [ ]:
# from torchvision import models 
# model = models.efficientnet_b0()

# #model.load_state_dict("/kaggle/input/timm-pretrained-efficientnet/efficientnet/efficientnet_b0_ra-3dd342df.pth")
# model.classifier = nn.Sequential(
#     nn.Linear(in_features=model.classifier.in_features, out_features=1)
# )
# #model

In [ ]:
model.load_state_dict(torch.load("/kaggle/input/effenetb0final/checknew.pth", map_location=device))
#model = torch.load("check.pth")
model.eval()

In [ ]:
#eff = '/kaggle/input/timm-pretrained-efficientnet/efficientnet/efficientnet_b0_ra-3dd342df.pth'

In [ ]:
# class MyEffNetB0Model(nn.Module):
#     def __init__(self):
#         super(MyEffNetB0Model, self).__init__()
        
#         # Load a pre-defined EfficientNet-B0 model
#         self.network = timm.create_model(eff, pretrained=False)
        
#         # Replace the last layer with your own output layer
#         num_ftrs = self.network.classifier.in_features
#         self.network.classifier = nn.Linear(num_ftrs, num_classes)

#     def forward(self, x):
#         # Define the forward pass of your model here
#         return self.network(x)

In [ ]:
import sys
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
sys.path.append("../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master")

In [ ]:
# from efficientnet_pytorch import model as enet
# from efficientnet_pytorch import EfficientNet

# # # Initialize the model
# model = EfficientNet.from_name('efficientnet-b0')

In [ ]:
# model._fc = nn.Sequential(
#     nn.Linear(in_features=model._fc.in_features, out_features=1)
# )

In [ ]:
# import joblib

# # Load the model from the file in the input directory
# model_path = "/kaggle/input/efficientnet-pytorch/efficientnet-b0-08094119.pth"
# model = joblib.load(model_path)

In [ ]:
def evaluate_model(model,val_dl):
    torch.manual_seed(42)
    pred_cancer = []
    with torch.no_grad():
        model.eval()
        losses = []
        targets = []
        with tqdm(val_dl,desc='Eval',mininterval=30) as eval_progress:
            for i,batch in enumerate(eval_progress):
                with autocast(enabled=True):
                    imgs,labels = batch
                    cancer_pred = model.forward(imgs)
                    loss = F.binary_cross_entropy_with_logits(
                            cancer_pred,
                            labels.to(float),
                            pos_weight = torch.tensor([20]).to(device)
                        ).item()
                    losses.append(loss)
                    pred_cancer.append(torch.sigmoid(cancer_pred))
                    targets.append(labels.cpu().numpy())
        targets = np.concatenate(targets)
        pred = torch.concat(pred_cancer).cpu().numpy()
    return np.mean(losses)

In [ ]:
!pip install timm -q

In [ ]:
#model

In [ ]:
#model.load_state_dict(torch.load("/kaggle/input/effenetb0final/checknew.pth", map_location=device))
#model = torch.load("check.pth")
#model.eval()

In [ ]:
test_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
train_df = pd.read_csv('/kaggle/input/rsna-datadrame/rsna_df.csv')

In [ ]:
PNG_TEST_IMAGES_PATH = f'/kaggle/working/test'
DCM_TEST_IMAGES_PATH = f'/kaggle/input/rsna-breast-cancer-detection/test_images'

In [ ]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import re
import pydicom
from tqdm import tqdm
import glob
import cv2
import os

def fit_image(fname, size=1024):
    # 1. Read, resize
    
    
    patient = fname.split('/')[-2]
    image = fname.split('/')[-1][:-4]
    dicom = pydicom.dcmread(fname)
    img = dicom.pixel_array
    img = (img - img.min()) / (img.max() - img.min())
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = 1 - img
    img = cv2.resize(img, (size, size))
    
    # 2. Crop
    X = img
    # Some images have narrow exterior "frames" that complicate selection of the main data. Cutting off the frame
    X = X[5:-5, 5:-5]
    
    
    # regions of non-empty pixels
    output= cv2.connectedComponentsWithStats((X > 0.05).astype(np.uint8)[:, :], 8, cv2.CV_32S)
    stats = output[2]
    
    # finding max area which always corresponds to the breast data. 
    idx = stats[1:, 4].argmax() + 1
    x1, y1, w, h = stats[idx][:4]
    x2 = x1 + w
    y2 = y1 + h
    
    # cutting out the breast data
    X_fit = X[y1: y2, x1: x2]    
    patient_id, im_id = os.path.basename(os.path.dirname(fname)), os.path.basename(fname)[:-4]
    os.makedirs(f'{PNG_TEST_IMAGES_PATH}/{patient_id}', exist_ok=True)
    cv2.imwrite(f'{PNG_TEST_IMAGES_PATH}/{patient_id}/{im_id}.png', (X_fit[:, :] * 255).astype(np.uint8))

def fit_all_images(all_images):
    with ThreadPoolExecutor(2) as p:
        for i in tqdm(p.map(fit_image, all_images), total=len(all_images)):
            pass

all_images = glob.glob('/kaggle/input/rsna-breast-cancer-detection/test_images/*/*') 
# all_images = glob.glob('/kaggle/input/rsna-breast-cancer-detection/train_images/10006/*')
fit_all_images(all_images)

In [ ]:
test_img_path ='/kaggle/working/test/'

In [ ]:
test_df

In [ ]:
test_df = train_df.copy()

In [ ]:
test_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')

In [ ]:
base_path = test_img_path
all_paths = []
for k in tqdm(range(len(test_df))):
    row = test_df.iloc[k, :]
    all_paths.append(base_path  +str(row.patient_id)+"/"+str(row.image_id) +".png")
    
test_df["path"] = all_paths

In [ ]:
test_df

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
path_test = test_df["path"] 

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
Target = 'cancer'
class RSNADataset(Dataset):
    def __init__(self, df, transforms=None):
        super().__init__()
        self.df = df
        self.img_path = list(df['path'])
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img = self.img_path[idx]
        img = Image.open(img).convert('RGB')
        if (self.transforms!=None):
            img = self.transforms(img)
        
        if Target in self.df.columns:
            target = self.df.loc[idx,'cancer']
            return img,torch.tensor(np.array([target]))
        return img

In [ ]:
import torchvision
from PIL import Image

def get_transforms(aug=False):

    def transforms(img):
#         img = img.convert('RGB')#.resize((512, 512))
        if aug:
            tfm = [
                torchvision.transforms.RandomHorizontalFlip(0.5),
                torchvision.transforms.RandomRotation(degrees=(-5, 5)), 
                torchvision.transforms.RandomResizedCrop((1024, 512), scale=(0.8, 1), ratio=(0.45, 0.55)) 
            ]
        else:
            tfm = [
                torchvision.transforms.RandomHorizontalFlip(0.5),
                torchvision.transforms.RandomVerticalFlip(0.5),
                torchvision.transforms.Resize((256,256))
            ]
        img = torchvision.transforms.Compose(tfm + [            
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            
        ])(img)
        return img
    return lambda img: transforms(img)

In [ ]:
test_df

In [ ]:
test_ds =RSNADataset(test_df,get_transforms(False))

In [ ]:
test_loader = DataLoader(test_ds, batch_size=16,shuffle=False,num_workers=2,pin_memory=True)
test_dl_final = DeviceDataLoader(test_loader,device)

In [ ]:
cnn_pred = []
for image in test_dl_final:
    
    pred = model(image).cpu().detach().numpy()
    # print(pred.cpu().detach().numpy())
    for i in range(len(pred)):
        cnn_pred.append(pred[i])

In [ ]:
cnn_pred

In [ ]:
ans = []
for i in range(len(cnn_pred)):
    x = torch.from_numpy(cnn_pred[i])
    m = nn.Sigmoid()
    z = m(x)
    ans.append(z)

In [ ]:
test_df

In [ ]:
df_test = test_df
df_test['cancer'] = ans
df_sub = df_test.groupby('prediction_id')[['cancer']].mean().reset_index()
df_sub

In [ ]:
THRES = .6
df_sub['cancer'] = (df_sub.cancer > THRES).astype(float)
df_sub

In [ ]:
df_sub.to_csv('submission.csv',index=False)